In [79]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


import numpy as np
import pandas as pd
from tqdm import tqdm
import refinitiv.data as rd
from refinitiv.data.content import news
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [83]:
constituents = pd.read_csv("constituents.csv")
rics = constituents["RIC"].tolist()
rics = rics[:50]

In [84]:
rd.open_session()

analyzer = SentimentIntensityAnalyzer()

all_sentiment = {}

for ric in tqdm(rics):
    try:
        headlines = news.headlines.Definition(query=f"RIC:{ric}", count=5000).get_data()
        df = headlines.data.df.reset_index()

        df["sentiment"] = df["headline"].apply(lambda t: analyzer.polarity_scores(t)["compound"])

        df["versionCreated"] = pd.to_datetime(df["versionCreated"])
        df["date"] = df["versionCreated"].dt.date

        daily_sent = df.groupby("date")["sentiment"].mean()

        all_sentiment[ric] = daily_sent

    except Exception as e:
        print(f"Error fetching {ric}: {e}")

sentiment_df = pd.DataFrame(all_sentiment)

sentiment_df.index = pd.to_datetime(sentiment_df.index)
sentiment_df = sentiment_df.sort_index()



sentiment_df

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [43:35<00:00, 52.32s/it]


,1COVG.DE,AALB.AS,ABB.ST,ABI.BR,ABNd.AS,ACCP.PA,ACKB.BR,AD.AS,ADP.PA,ADSGn.DE,...,AXAF.PA,AYV.PA,AZN.ST,BAKKA.OL,BAMN.AS,BAR.BR,BASFn.DE,BAYGn.DE,BEIG.DE,BESI.AS
date,,,,,,,,,,,,,,,,,,,,,
2024-08-29,NaN,0.000000,NaN,NaN,NaN,NaN,-0.059200,NaN,0.000000,NaN,...,NaN,NaN,NaN,0.000000,0.458800,0.000000,NaN,NaN,NaN,NaN
2024-08-30,NaN,0.097143,NaN,NaN,NaN,NaN,0.118072,NaN,0.057350,NaN,...,NaN,NaN,NaN,0.048571,0.123475,-0.092050,NaN,NaN,NaN,NaN
2024-08-31,NaN,0.190900,NaN,NaN,NaN,NaN,0.202300,NaN,0.229400,NaN,...,NaN,0.458800,NaN,0.399400,0.229400,-0.220200,NaN,NaN,NaN,NaN
2024-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.680800,NaN,...,NaN,-0.636900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-09-02,NaN,0.204000,NaN,NaN,NaN,NaN,0.069858,NaN,0.068000,NaN,...,NaN,NaN,NaN,0.080000,0.118400,0.115920,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-25,-0.007857,0.092103,-0.015525,-0.008350,0.005826,0.117907,0.001700,0.005856,-0.065113,-0.004721,...,0.068667,-0.207750,0.099689,-0.085047,0.057038,-0.034410,0.038133,0.087268,0.075463,0.046695
2025-11-26,-0.012454,0.019342,-0.002920,0.022997,0.050397,0.173255,0.047475,0.056558,-0.078214,0.051249,...,0.098396,-0.239050,0.064963,0.130924,0.137097,-0.054640,0.047170,0.042662,0.042269,0.111430
2025-11-27,0.029329,0.224224,0.017276,0.140642,0.112219,0.087937,-0.090461,0.221857,-0.078214,-0.016796,...,0.289725,-0.227667,0.081952,0.007987,0.175743,-0.071122,0.111196,0.067215,-0.093400,0.183973
